<a href="https://colab.research.google.com/github/cardonasebastian/MNA_PROYECTO/blob/main/Avance4/Modelos_alternativos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTITUTO TECNOLÓGICO DE ESTUDIOS SUPERIORES DE MONTERREY

# Maestría en Inteligencia Artificial Aplicada

# TC5035.10 Proyecto Integrador:

# Optimización Estratégica de la Seguridad Vial: Implementación de un Modelo de Clasificación Avanzado para la Evaluación y Mejora de Hábitos de Conducción


#Fase 4.
# Avance 4. Modelos Alternativos

#A01687530 - Fernando Sebastian Sanchez Cardona
#A01793670 - Bryan Alvarado Cruz
# A01793832 - Fernando Anaya Delgado

# Lee los datos safely Gold

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
file_path = '/content/drive/My Drive/MNA_PI/Gold/safety.csv'
df_safety = pd.read_csv(file_path)
df_safety

,unique_id,date,name,maxSpeed,hard_acceleration,harsh_braking,harsh_cornering,seat_belt,speeding,excessive_speeding
0,74606d62-df5f-4cf4-9ba5-262c1c64731f,2024-05-17,1,112,0,0,0,0,0,0
1,5414fa29-5a8b-402e-ba0c-65877396a103,2024-05-17,2021,114,0,0,0,0,0,0
2,02b64477-38b6-47d4-ab22-fe50d5253175,2024-05-17,2022,112,0,1,0,0,0,0
3,23ef12be-25c4-43cf-9489-5b2345fe415a,2024-05-17,2023,12,0,0,0,0,2,0
4,64adf191-c891-43d9-9ba0-3819652e285f,2024-05-17,2024,10,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
4337,4635d0c2-affb-4a62-bc95-5e92560034c0,2024-04-27,2212,122,0,12,2,7,40,3
4338,1568b4e9-172e-49e7-a12c-d3f964213804,2024-04-27,2213,0,0,0,0,0,0,0
4339,b75f5269-c9b8-41ce-b8c5-1764e18b5be7,2024-04-27,2214,116,0,0,0,0,0,0
4340,ebbec643-0dbd-4350-903e-e197e7c81284,2024-04-27,2215,118,0,2,0,0,2,0


# Tomando en cuenta el script anterior de BaseLine

### Se importan las librerias

In [7]:
# Importar las bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score

### Se carga el dataset y se limpia

In [8]:


# Cargar el dataset
df = df_safety
# Eliminar columnas innecesarias
df_limpio = df.drop(columns=['unique_id', 'date', 'name'])

# Crear una variable de riesgo basada en comportamientos de conducción
df_limpio['puntuacion_riesgo'] = df_limpio[['hard_acceleration', 'harsh_braking', 'harsh_cornering', 'seat_belt', 'speeding', 'excessive_speeding']].sum(axis=1)



### Se define la funcion para clasificar al conductor

In [9]:
# Definir la variable objetivo basada en la puntuación de riesgo
def clasificar_conductor(puntuacion_riesgo):
    if puntuacion_riesgo == 0:
        return 0  # Buen conductor
    elif puntuacion_riesgo <= 3:
        return 1  # Conductor regular
    else:
        return 2  # Mal conductor

df_limpio['clase_conductor'] = df_limpio['puntuacion_riesgo'].apply(clasificar_conductor)

# Eliminar la columna de puntuación de riesgo
df_limpio = df_limpio.drop(columns=['puntuacion_riesgo'])



### Se definen caracteristicas y objetivos

In [10]:
# Definir características y objetivo
X = df_limpio.drop(columns=['clase_conductor'])
y = df_limpio['clase_conductor']



### Se dividen los datos de prueba y validacion

In [11]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



### Se normalizan los datos

In [12]:
# Normalizar las características numéricas
escalador = StandardScaler()
X_train_escalado = escalador.fit_transform(X_train)
X_test_escalado = escalador.transform(X_test)



### Se definen los modelos a evaular y comparar

In [13]:
# Definir los modelos
modelos = {
    'Regresión Logística': LogisticRegression(max_iter=1000, random_state=42, multi_class='ovr'),
    'Árbol de Decisión': DecisionTreeClassifier(random_state=42),
    'Bosque Aleatorio': RandomForestClassifier(random_state=42),
    'Máquina de Vectores de Soporte': SVC(random_state=42),
    'K-Vecinos Más Cercanos': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Red Neuronal': MLPClassifier(max_iter=1000, random_state=42),
    'XGBClassifier': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    'KNN': KNeighborsClassifier()
}



### Se entrenan los modelos

In [14]:
# Entrenar y evaluar los modelos
rendimiento_modelos = {}

for nombre_modelo, modelo in modelos.items():
    modelo.fit(X_train_escalado, y_train)
    y_pred = modelo.predict(X_test_escalado)
    precision = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    rendimiento_modelos[nombre_modelo] = {'Precisión': precision, 'F1-Score': f1}



### Se muestran los resultados de los entrenamientos de los modelos

In [15]:
# Convertir los resultados a un DataFrame para la comparación
rendimiento_df = pd.DataFrame(rendimiento_modelos).T
rendimiento_df = rendimiento_df.sort_values(by='F1-Score', ascending=False)

print("Comparación del rendimiento de los modelos:")
print(rendimiento_df)



Comparación del rendimiento de los modelos:
                                Precisión  F1-Score
Red Neuronal                     0.998849  0.998845
Bosque Aleatorio                 0.995397  0.995364
XGBClassifier                    0.994246  0.994266
Árbol de Decisión                0.988493  0.988649
K-Vecinos Más Cercanos           0.979287  0.979079
KNN                              0.979287  0.979079
Máquina de Vectores de Soporte   0.959724  0.961108
Regresión Logística              0.947066  0.945769
Naive Bayes                      0.937860  0.945037


### Se ajustan los hiperametros de los modelos

In [16]:
# Ajuste de Hiperparámetros para los Mejores Modelos

# XGBClassifier
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid_xgb, scoring='f1_weighted', cv=5, n_jobs=-1)
grid_search_xgb.fit(X_train_escalado, y_train)

# Mejor modelo XGB
mejor_xgb = grid_search_xgb.best_estimator_
mejor_xgb_pred = mejor_xgb.predict(X_test_escalado)
mejor_xgb_f1 = f1_score(y_test, mejor_xgb_pred, average='weighted')
print("Mejor F1-Score para XGBClassifier:", mejor_xgb_f1)

# Bosque Aleatorio
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, scoring='f1_weighted', cv=5, n_jobs=-1)
grid_search_rf.fit(X_train_escalado, y_train)

# Mejor modelo Bosque Aleatorio
mejor_rf = grid_search_rf.best_estimator_
mejor_rf_pred = mejor_rf.predict(X_test_escalado)
mejor_rf_f1 = f1_score(y_test, mejor_rf_pred, average='weighted')
print("Mejor F1-Score para Bosque Aleatorio:", mejor_rf_f1)

# Selección del Modelo Final
if mejor_xgb_f1 > mejor_rf_f1:
    modelo_final = mejor_xgb
    print("El modelo final seleccionado es XGBClassifier con un F1-Score de:", mejor_xgb_f1)
else:
    modelo_final = mejor_rf
    print("El modelo final seleccionado es Bosque Aleatorio con un F1-Score de:", mejor_rf_f1)

Mejor F1-Score para XGBClassifier: 0.9953639969666919
Mejor F1-Score para Bosque Aleatorio: 0.9906595082952482
El modelo final seleccionado es XGBClassifier con un F1-Score de: 0.9953639969666919


# Conclusion

### Después de entrenar y evaluar múltiples modelos de clasificación utilizando el dataset de comportamientos de conducción, el modelo XGBClassifier se destacó como el mejor en términos de rendimiento, específicamente en su F1-Score. Aquí se presentan las razones clave que explican por qué XGBClassifier superó a los demás modelos:

##	1.	Capacidad para Manejar Datos No Lineales:
###		XGBClassifier es conocido por su capacidad para capturar relaciones no lineales en los datos debido a su naturaleza basada en árboles. A diferencia de modelos más simples como la Regresión Logística, XGBClassifier puede modelar interacciones complejas entre características, lo que resulta en un mejor rendimiento en datasets con estructuras intrincadas.
##	2.	Control del Sobreajuste:
###		XGBClassifier incorpora técnicas avanzadas para controlar el sobreajuste, como la regularización y el uso de múltiples parámetros de ajuste.Esto permite al modelo generalizar mejor en datos no vistos, como se observó en el conjunto de prueba.
##	3.	Boosting Gradual:
###		El enfoque de boosting gradual utilizado por XGBClassifier combina múltiples árboles de decisión, donde cada árbol sucesivo corrige los errores del anterior. Esta técnica de mejora continua hace que el modelo sea muy eficaz para reducir errores y mejorar la precisión general.
##	4.	Flexibilidad y Personalización:
###		XGBClassifier ofrece una amplia gama de hiperparámetros ajustables, permitiendo una personalización precisa del modelo para optimizar el rendimiento. El proceso de ajuste de hiperparámetros mediante GridSearchCV permitió encontrar la configuración óptima que maximiza el F1-Score.
##	5.	Manejo de Datos Desequilibrados:
###		En problemas de clasificación con clases desbalanceadas, como el presente dataset, XGBClassifier tiene opciones para ajustar el balance de las clases mediante el parámetro scale_pos_weight, lo que ayuda a mejorar la sensibilidad del modelo hacia las clases minoritarias.
##	6.	Velocidad y Eficiencia:
###		A pesar de ser un algoritmo complejo, XGBClassifier es muy eficiente en términos de tiempo de entrenamiento gracias a su implementación optimizada. Esto lo hace adecuado para datasets grandes y complejos sin sacrificar la velocidad de entrenamiento.

#Comparación con Otros Modelos:

###		Regresión Logística: Es un modelo lineal que puede no capturar las complejidades y no linealidades del dataset. Aunque es fácil de interpretar, su rendimiento es inferior.
###		Árbol de Decisión y K-Vecinos Más Cercanos: Pueden ser propensos al sobreajuste y no siempre manejan bien la complejidad de los datos.
###		Bosque Aleatorio: También basado en árboles y robusto, pero generalmente no supera a XGBClassifier en datasets complejos debido a la falta de boosting.
###		Máquina de Vectores de Soporte: Potente en alta dimensionalidad pero puede ser más lento y menos flexible en la gestión de hiperparámetros en comparación con XGBClassifier.
###		Naive Bayes: Asume independencia entre características, lo cual rara vez es cierto en problemas reales, afectando su rendimiento.

#Conclusión Final:

## El modelo XGBClassifier es superior debido a su capacidad para manejar datos complejos y no lineales, su robustez contra el sobreajuste, y su flexibilidad y eficiencia en el ajuste de hiperparámetros. Estas ventajas permiten que el modelo alcance un F1-Score más alto y una mejor generalización en comparación con otros algoritmos.